In [1]:
%pwd

'd:\\MLOPS\\TextSummrizer\\research'

In [2]:
import os
os.chdir("../")
%pwd

'd:\\MLOPS\\TextSummrizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int
    predict_with_generate: bool # High-value addition for Seq2Seq


In [4]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig: 
        config=self.config.model_trainer
        params=self.params.Seq2SeqTrainingArguments

        create_directories([config.root_dir])   
        model_trainer_config=ModelTrainerConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                model_ckpt=config.model_ckpt,
                num_train_epochs=params.num_train_epochs,
                warmup_steps=params.warmup_steps,
                per_device_train_batch_size=params.per_device_train_batch_size,
                per_device_eval_batch_size=params.per_device_eval_batch_size,
                weight_decay=params.weight_decay,
                logging_steps=params.logging_steps,
                eval_strategy=params.eval_strategy,
                eval_steps=params.eval_steps,
                save_steps=params.save_steps,
                gradient_accumulation_steps=params.gradient_accumulation_steps,
                predict_with_generate=params.predict_with_generate
                )

        return model_trainer_config        

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
import torch ,accelerate
from datasets import load_from_disk

d:\MLOPS\TextSummrizer\summrizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config=config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"    
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading the data
        dataset_samsum_pt=load_from_disk(self.config.data_path)

        trainer_args = Seq2SeqTrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=500,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            weight_decay=0.01,
            logging_steps=10,
            eval_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=16,
            predict_with_generate=True # High-value addition for Seq2Seq
        )

        trainer = Seq2SeqTrainer(
        model=model_pegasus,
        args=trainer_args,
        data_collator=seq2seq_data_collator,
        train_dataset=dataset_samsum_pt["test"],
        eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()
        
        ## Save Model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [8]:
config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer=ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2026-02-12 12:27:51,583: INFO: common : yaml file: config\config.yaml loaded successfully]
[2026-02-12 12:27:51,591: INFO: common : yaml file: params.yaml loaded successfully]
[2026-02-12 12:27:51,594: INFO: common : created directory at: artifacts]
[2026-02-12 12:27:51,596: INFO: common : created directory at: artifacts/model_trainer]
[2026-02-12 12:27:52,055: INFO: _client : HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-12 12:27:52,110: INFO: _client : HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"]
[2026-02-12 12:27:52,422: INFO: _client : HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"]


[2026-02-12 12:27:52,426: WARNING: _http : Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.]
[2026-02-12 12:27:52,479: INFO: _client : HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/tokenizer_config.json "HTTP/1.1 200 OK"]
[2026-02-12 12:27:52,790: INFO: _client : HTTP Request: GET https://huggingface.co/api/models/google/pegasus-cnn_dailymail/tree/main/additional_chat_templates?recursive=false&expand=false "HTTP/1.1 404 Not Found"]
[2026-02-12 12:27:53,091: INFO: _client : HTTP Request: GET https://huggingface.co/api/models/google/pegasus-cnn_dailymail/tree/main?recursive=true&expand=false "HTTP/1.1 200 OK"]
[2026-02-12 12:27:53,760: INFO: _client : HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-12 12:27:53,811: INFO:

Loading weights: 100%|██████████| 680/680 [00:01<00:00, 411.44it/s, Materializing param=model.shared.weight]                                   
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
PegasusForConditionalGeneration LOAD REPORT from: google/pegasus-cnn_dailymail
Key                                  | Status  | 
-------------------------------------+---------+-
model.decoder.embed_positions.weight | MISSING | 
model.encoder.embed_positions.weight | MISSING | 

Notes:
- MISSING	:th

[2026-02-12 12:28:18,064: INFO: _client : HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/generation_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-12 12:28:18,096: INFO: _client : HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/generation_config.json "HTTP/1.1 200 OK"]


d:\MLOPS\TextSummrizer\summrizer\lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [19]:
import sys
print(sys.executable)


d:\MLOPS\TextSummrizer\summrizer\python.exe
